In [294]:
import numpy as np
import cv2

In [295]:
without_mask=np.load('without_the_mask.npy')
with_mask=np.load('with_the_mask.npy')


In [296]:
with_mask.shape


(200, 50, 50, 3)

In [297]:
#Converting to 2D
without_mask=without_mask.reshape(200,50*50*3)
with_mask=with_mask.reshape(200,50*50*3)


In [298]:
with_mask.shape


(200, 7500)

In [299]:
without_mask.shape

(200, 7500)

In [300]:
#Combing the 2 datasets with and without mask into 1
X=np.r_[with_mask,without_mask]

In [301]:
X.shape

(400, 7500)

In [302]:
#Giving 0 as label for mask and 1 for no mask
labels=np.zeros(X.shape[0])

In [303]:
labels[200:]=1.0

In [304]:
#KNN Machine Learning Model
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()

In [305]:
from sklearn.metrics import accuracy_score

In [306]:
#Partitioning the dataset
from sklearn.model_selection import train_test_split

In [307]:
X_train,X_test,Y_train,Y_test=train_test_split(X,labels,test_size=0.30,random_state=6)

In [308]:
X_train.shape

(280, 7500)

In [309]:
#Dimensionality Reduction
from sklearn.decomposition import PCA

In [310]:
pca=PCA(n_components=3)

In [311]:
X_train=pca.fit_transform(X_train)

In [312]:
X_train.shape

(280, 3)

In [313]:
X_test=pca.transform(X_test)

In [314]:
X_test

array([[-3.26018724e+03, -6.61131829e+01,  5.59445396e+02],
       [-3.27703151e+03, -3.60352415e+01,  5.30668061e+02],
       [-1.93718258e+03, -1.61491298e+02,  8.68144819e+01],
       [-2.89973863e+03, -4.33390639e+02,  2.05526507e+02],
       [-3.24033118e+03,  3.94341032e+01,  5.60272283e+02],
       [ 2.37192574e+03, -9.58892788e+02, -4.43218828e+02],
       [-2.65503567e+03, -1.83944836e+02,  2.36164701e+02],
       [ 1.99629288e+03, -8.39918642e+02,  1.94065622e+02],
       [ 2.35864585e+03, -1.18731646e+03, -3.07979413e+02],
       [ 2.31137216e+03, -8.89620004e+02, -8.28512048e+01],
       [-3.07173561e+03, -3.74873946e+02,  2.48708941e+02],
       [ 2.32146508e+03, -1.00449162e+03, -1.07167729e+02],
       [ 2.35811758e+03, -9.72146474e+02, -1.07260805e+02],
       [-3.25109977e+03, -1.00903595e+01,  5.50364943e+02],
       [-1.53866026e+03, -1.01590480e+02,  3.20920927e+01],
       [-3.11449608e+03, -3.57642477e+02,  3.43348123e+02],
       [ 2.43194805e+03, -1.04856271e+03

In [315]:
#Normalisation
from sklearn.preprocessing import StandardScaler
std=StandardScaler()

In [316]:
X_train=std.fit_transform(X_train)
X_test=std.transform(X_test)

In [317]:
X_train

array([[-1.04924342e+00, -2.35624290e-01, -1.23243575e-02],
       [ 8.93318306e-01, -6.59311328e-01, -4.38750943e-01],
       [ 9.21051684e-01, -6.51578548e-01, -3.88780655e-01],
       [ 8.83896294e-01, -6.31262261e-01, -9.29651025e-02],
       [-1.23037553e+00, -1.19444661e-01,  4.23583790e-01],
       [-1.11401823e+00, -2.31257725e-01,  7.17664603e-02],
       [-1.09753813e+00,  2.15371800e-01,  3.68372820e-01],
       [-1.05260192e+00, -3.60730281e-01,  2.78168646e-02],
       [-1.09424354e+00, -7.11921771e-02, -1.47430338e-01],
       [ 8.61188116e-01, -6.41627112e-01, -1.64125073e-01],
       [-1.15823473e+00, -2.17962307e-01,  3.46075838e-01],
       [ 9.30137599e-01, -6.17659537e-01, -2.99279160e-01],
       [ 1.72214352e+00,  2.41955988e+00,  4.68569653e+00],
       [ 9.18215749e-01, -5.80176713e-01, -2.87829293e-01],
       [ 7.95463192e-01, -5.26648727e-01, -2.38792407e-01],
       [ 8.24666311e-01, -5.72576891e-01,  5.14739623e-02],
       [-1.25238974e+00, -4.44961133e-02

In [318]:
#Training
knn.fit(X_train,Y_train)

KNeighborsClassifier()

In [319]:
#Mapping
mask_unmask = {0: 'With Mask', 1:'No Mask'}

In [326]:
#Live Detection of whether the person is wearing mask or not
haar_data=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
capture=cv2.VideoCapture(0)
while True:
    flag,img=capture.read()
    if flag:
        faces=haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),4)
            face=img[y:y+h, x:x+w, :]
            face=cv2.resize(face,(50,50))
            face=face.reshape(1,-1)
            face=pca.transform(face)
            #face=std.transform(face)
            prediction=knn.predict(face)
            n=mask_unmask[int(prediction)]
            print(n)
        cv2.imshow("Face",img)
        if cv2.waitKey(2) == 27:
            break
capture.release()
cv2.destroyAllWindows()

No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
